# Launch Site Location Analysis w/ Folium

In this notebook, we will be exploring how the location and proximities of a launch site affects the success rate of the launch.</br> This analysis aims to identify the factors which can help us determine an optimal launch site location.
____

Importing our libraries:

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

Loading our data:

In [2]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df.head(2)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356


### 1. Marking launch sites on our map

In [3]:
# defining a useful df
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Creating our map with our initial center location being NASA Johnson Space Center at Houston, Texas.

In [4]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Adding marker and circle identifying NASA JSC.

In [5]:
# creating blue circle at NASA JSC coordinates with popup
circle = folium.Circle(nasa_coordinate, radius=1000, color='#065D92 ', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# creating marker labeling our circle 
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#065D92 ;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Adding circle and marker for each launch site

In [6]:
# initializing our map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [7]:

for index, row in launch_sites_df.iterrows():
    coordinates = [row['Lat'], row['Long']]
    circle = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
    coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [8]:
site_map

#### Inferences:
1. Launch sites are not placed near the Equator line.
2. Launch sites are in very close proximity to the coasts.
    - Further, they are all relatively close to the southern portion of their respective coast.

### 2. Marking the succesful/failed launches for each site on the map
Our dataframe, has a column class representing the outcome of each launch. We will mark these on the map by using green for succesful (`1`) and red for failed (`0`).

We must consider however, that launches only originate out of four possible site locations, this means that they will have the same coordinates. We will use a marker cluster to mitigate this.

In [9]:
marker_cluster = MarkerCluster()

Creating a new column to represent our marker colors and assiging value:

In [10]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

Adding markers to our map:

In [11]:
site_map.add_child(marker_cluster)
for index, row in spacex_df.iterrows():
    coordiates = [row['Lat'], row['Long']]
    marker = folium.map.Marker(
        coordiates,
        icon=folium.Icon(
            color='white',
            icon_color=row['marker_color']
        )
    )
    marker_cluster.add_child(marker)

site_map

### 3. Calculating the distances between a launch site to it's proximities

First, let's add a `MousePosition` to get coordinates when exploring the map.

In [12]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

Defining function to help us calculate distance:

In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Calculating distance between launch site(KSC LC-39A) and coastline:

In [14]:
# coast coordinates
coast_lat = 28.5627
coast_long = -80.56768
# KSC LC-39A coordintaes
ksc_lat = 28.573255
ksc_long = -80.646895
distance = calculate_distance(ksc_lat, ksc_long, coast_lat, coast_long)
print(distance, ' km')

7.826879668745357  km


Adding marker to our map showing this distance:

In [15]:
coordinates = [coast_lat, coast_long]
distance_marker = folium.map.Marker(
    coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    )
)
site_map.add_child(distance_marker)


Adding `PolyLine` from launch site to coast:

In [16]:
ksc_coord = [ksc_lat,ksc_long]
coast_coord = [coast_lat, coast_long]
coordinates = [ksc_coord, coast_coord]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

#### Adding marker and line with distance to closest city, Melbourne

Calculating distance:

In [17]:
melbourne_lat = 28.10542
melbourne_long = -80.63656
distance = calculate_distance(ksc_lat, ksc_long, melbourne_lat, melbourne_long)
print(distance, ' km')

52.04704436043004  km


Adding marker:

In [18]:
melbourne_coord = [melbourne_lat, melbourne_long]
distance_marker = folium.map.Marker(
    melbourne_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance)
    )
)
site_map.add_child(distance_marker)

Adding line:

In [19]:
coordinates = [ksc_coord, melbourne_coord]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

#### Inferences:
1. Launch sites tend to be closer to coastlines.
2. Launch sites tend to be far from major citites.